on this file we will create one CSV file from all the cv we have scrapped. Also columns that can be deleted will be deleted (see analysis) 

In [1]:
import os
import warnings
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# panda data frame containing every infos
col_names = [
    "Locality",
    "Type_of_property",
    "Subtype_of_property",
    "Price",
    "Type_of_sale",
    "Number_of_rooms",
    "Living_Area",
    "Fully_equipped_kitchen",
    "Furnished",
    "Open_fire",
    "Terrace",
    "Terrace_Area",
    "Garden",
    "Garden_Area",
    "Surface_of_the_land",
    "Surface_area_of_the_plot_of_land",
    "Number_of_facades",
    "Swimming_pool",
    "State_of_the_building",
]
df_final = pd.DataFrame(columns=col_names)

In [4]:
def create_df():
    """
    create a dataframe from all the csv we created
    """
    # List all files in the directory
    global df_final
    df_final = pd.DataFrame(columns=col_names)

    # path = "./all_csv"
    path = r"C:\Users\Sacha\Documents\BeCode\real-estate-price-prediction\data_acquisition\all_csv"
    files = os.listdir(path)

    for file in files:
        if os.path.isfile(os.path.join(path, file)):
            file_path = (
                r"C:\Users\Sacha\Documents\BeCode\real-estate-price-prediction\data_acquisition\all_csv\\"
                + file
            )
            file_df = pd.read_csv(file_path)
            with warnings.catch_warnings():  # warnings from panda because columns with only bool
                warnings.simplefilter("ignore")
                df_final = pd.concat([df_final, file_df])

In [5]:
def del_columns():
    """
    this function clean the data a first time, it will delete useless lines or columns
    """
    global df_final
    # remove the useless collumn
    del df_final["Surface_of_the_land"]

    # this line will drop lines where Price or the living area is missing
    df_final = df_final.dropna(subset=["Price"])
    df_final = df_final.dropna(subset=["Living_Area"])

    # drop lines full of nan
    df_final = df_final.dropna(how="all")

    # this collumn is the indexes of the dataframe collumn (too much)
    df_final = df_final.drop(df_final.columns[-1], axis=1)

    # we only keep residential sales
    df_final = df_final[df_final["Type_of_sale"] == "residential_sale"]
    del df_final["Type_of_sale"]

    # drop this column, too correlated
    del df_final["Surface_area_of_the_plot_of_land"]

    # Garden and terrace are only nan or true, no false and information in the garden/terrace area
    del df_final["Garden"]
    del df_final["Terrace"]
    df_final["Garden_Area"] = df_final["Garden_Area"].fillna(0)
    df_final["Terrace_Area"] = df_final["Terrace_Area"].fillna(0)

    # drop collumns that seems useless
    del df_final["Fully_equipped_kitchen"]
    del df_final["Furnished"]

    # remove_duplicates
    df_final = df_final.drop_duplicates()

In [6]:
def save_df():
    """
    save tthe global df into a csv
    """
    global df_final
    df_final.to_csv("ready_to_learn_csv.csv", index=False)

In [7]:
create_df()
del_columns()
save_df()